### Search Engine With Tools And Agents

LLMs cannot directly:

Call HTTP APIs

Parse unpredictable API responses

Handle rate limits or parameters safely

Wrappers solve this by:

Exposing simple Python methods

Returning clean text the LLM can reason over

Making tools agent-friendly

In [1]:
## Arxiv--Research
## Tools creation
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

In [2]:
# use the inbuilt tool of wikipedia
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
# then run the wrapper
wiki = WikipediaQueryRun(api_wrapper = api_wrapper_wiki)
wiki.name

'wikipedia'

In [3]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results = 1, doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv.name

'arxiv'

In [4]:
tools = [wiki, arxiv]

In [5]:
# Custom tools [RAG Tools]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

c:\Users\sanaz.rabinia\Gen_AI_Udemy\python\Question_Answer_Chatbot\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OpenAIEmbeddings())
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000027EDCB4B730>, search_kwargs={})

#### Convert the retriever into tools

In [7]:
from langchain_core.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith-search","Search any information about Langsmith ")

retriever_tool.name

'langsmith-search'

In [8]:
tools=[wiki,arxiv,retriever_tool]

In [9]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\sanaz.rabinia\\Gen_AI_Udemy\\python\\Question_Answer_Chatbot\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 StructuredTool(name='langsmith-search', description='Search any information about Langsmith ', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=<function create_retriever_tool.<locals>.func at 0x0000027EDCB8B7F0>, coroutine=<function create_retriever_tool.<locals>.afunc at 0x0000027EDCA8E680>)]

#### Run all this tools with Agents and LLM Models

In [ ]:
## Tools, LLM-->AgentExecutor
import openai
import os
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()


groq_api_key=os.getenv("GROQ_API_KEY") 
openai.api_key=os.getenv("OPENAI_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="openai/gpt-oss-120b")

#### Prompt Template

In [11]:

from langchain_classic import hub
prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

#### Agents

In [12]:
# this is one of the ways to create agents
from langchain_classic.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

#### Agent Executer to run the agent

In [13]:
from langchain_classic.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [14]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'Langsmith'}`


LangSmith is framework agnostic, so you can use it with or without LangChain’s open-source libraries
langchain and langgraph.
Prototype locally, then move to production with integrated monitoring and evaluation to build more reliable AI systems.
LangGraph Platform is now LangSmith Deployment. For more information, check out the Changelog.
​Get started
Create an accountSign up at smith.langchain.com (no credit card required).
You can log in with Google, GitHub, or email.Create an API keyGo to your Settings page → API Keys → Create API Key.
Copy the key and save it securely.
Once your account and API key are ready, choose a quickstart to begin building with LangSmith:

LangSmith docs - Docs by LangChainSkip to main contentDocs by LangChain home pageLangSmithSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationLangSmith docsGet startedObservabilityEvaluationPrompt engineeri

{'input': 'Tell me about Langsmith',
 'output': '**LangSmith – Overview**\n\nLangSmith is a **full‑stack observability, evaluation, and deployment platform for building and running LLM‑powered applications**. It is created by the team behind **LangChain** (and its newer graph‑based counterpart LangGraph), but it is **framework‑agnostic**—you can use it with LangChain, LangGraph, or any other LLM‑orchestration library.\n\n---\n\n### Core Capabilities\n\n| Area | What LangSmith Provides |\n|------|--------------------------|\n| **Observability** | End‑to‑end tracing of every request that hits your LLM pipeline (inputs, outputs, intermediate steps, latency, token usage, etc.). |\n| **Evaluation** | Built‑in metrics and custom evaluation functions to score responses (e.g., factuality, relevance, safety). You can run batch evaluations and see visual dashboards. |\n| **Prompt Engineering** | Version‑controlled prompt editing, collaborative review, and “prompt testing” that lets you iterate q

In [15]:
agent_executor.invoke({"input":"What is machine learning"})



> Entering new AgentExecutor chain...
**Machine learning (ML)** is a sub‑field of artificial intelligence (AI) that focuses on developing algorithms and statistical models that enable computers to *learn* from data, rather than being explicitly programmed for every task. In other words, instead of writing a set of hard‑coded rules, we give a system examples (training data) and let it infer the underlying patterns, relationships, or decision boundaries that it can then apply to new, unseen data.

---

## Core Idea

1. **Data → Model → Prediction**  
   - **Data**: A collection of examples (e.g., images, text, sensor readings) often accompanied by labels (e.g., “cat” vs. “dog”).  
   - **Model**: A parameterized mathematical function (e.g., a linear equation, a neural network) that maps inputs to outputs.  
   - **Learning**: An optimization process that adjusts the model’s parameters to minimize a loss (error) on the training data.  
   - **Prediction/Inference**: Once trained, the mo

{'input': 'What is machine learning',
 'output': '**Machine learning (ML)** is a sub‑field of artificial intelligence (AI) that focuses on developing algorithms and statistical models that enable computers to *learn* from data, rather than being explicitly programmed for every task. In other words, instead of writing a set of hard‑coded rules, we give a system examples (training data) and let it infer the underlying patterns, relationships, or decision boundaries that it can then apply to new, unseen data.\n\n---\n\n## Core Idea\n\n1. **Data → Model → Prediction**  \n   - **Data**: A collection of examples (e.g., images, text, sensor readings) often accompanied by labels (e.g., “cat” vs. “dog”).  \n   - **Model**: A parameterized mathematical function (e.g., a linear equation, a neural network) that maps inputs to outputs.  \n   - **Learning**: An optimization process that adjusts the model’s parameters to minimize a loss (error) on the training data.  \n   - **Prediction/Inference**: 

In [16]:
agent_executor.invoke({"input":"What's the paper 1706.03762 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on c
Invoking: `wikipedia` with `{'query': 'Attention Is All You Need'}`


Page: Attention Is All You Need
Summary: "Attention Is All You Need" is a 2017 landmark research paper in machine learning authored by eight scientists working at Google. The paper introduced a new deep learning architecture known as the transformer,**Paper:** *Attention Is All You Need* (arXiv:1706.03762)  
**Authors:** Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, Illia Polosukhin  
**Published:** June 2017 (Conference: NIPS 2017)  

---

### 1. Core Idea
The paper introduces the **Transformer** architecture—a neural netw

{'input': "What's the paper 1706.03762 about?",
 'output': '**Paper:** *Attention Is All You Need* (arXiv:1706.03762)  \n**Authors:** Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N.\u202fGomez, Łukasz\u202fKaiser, Illia\u202fPolosukhin  \n**Published:** June\u202f2017 (Conference: NIPS 2017)  \n\n---\n\n### 1. Core Idea\nThe paper introduces the **Transformer** architecture—a neural network for sequence‑to‑sequence tasks that **relies solely on attention mechanisms** and discards recurrent (RNN/LSTM) and convolutional layers. By doing so, it enables **full parallelism** over input and output positions, dramatically speeding up training and inference while achieving superior performance.\n\n---\n\n### 2. Architecture Overview  \n\n| Component | Description |\n|-----------|-------------|\n| **Multi‑Head Self‑Attention** | Each token attends to every other token in the same sequence, using several parallel attention “heads” to capture different relational